In [6]:
# run_cnn_baselines.py (수정 버전)
# PyTorch CNN baselines for text classification
# Fix: masking, submission format, preprocessing consistency

import os, math, random, numpy as np, pandas as pd, torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, classification_report
from transformers import (
    AutoTokenizer, AutoModel, AutoConfig, set_seed
)

# ---------------- CFG ----------------
class CFG:
    model_type   = "bertcnn"          # "bertcnn" or "textcnn"
    bert_name    = "beomi/KcBERT-base"  # 🔥 수정: KcBERT 
    max_len      = 384                # 🔥 수정: 512 → 384 (안정적)
    batch_size   = 16
    epochs       = 10                 # 🔥 수정: 5 → 10
    lr           = 2e-5
    lr_textcnn   = 1e-3
    weight_decay = 0.01
    val_ratio    = 0.15
    seed         = 2025
    data_dir     = "data"
    out_dir      = "outputs"
    device       = "cuda" if torch.cuda.is_available() else "cpu"
    
    # 🔥 추가 옵션
    use_class_weight = True           # 클래스 가중치
    early_stopping_patience = 4

os.makedirs(CFG.out_dir, exist_ok=True)
set_seed(CFG.seed)
device = torch.device(CFG.device)


# --------- IO ----------
def load_train_df(path):
    df = pd.read_csv(path)
    df = df.rename(columns={"conversation":"text","class":"label"})
    
    
    df["text"] = df["text"].astype(str).str.replace("\n", "[SEP]")
    df["text"] = df["text"].str.replace(r'\s+', ' ', regex=True)  # 연속 공백 제거
    df["text"] = df["text"].str.strip()
    
    df["label"] = df["label"].astype(str)
    return df

def load_test_df(path):
    df = pd.read_csv(path)
    # 원본 보존 (제출용)
    df["text_original"] = df["conversation"].astype(str)
    
    # 모델 입력용 전처리 (train과 동일)
    df["text"] = df["conversation"].astype(str).str.replace("\n", " ")
    df["text"] = df["text"].str.replace(r'\s+', ' ', regex=True)
    df["text"] = df["text"].str.strip()
    
    return df

# 제출 코드 매핑
NAME2CODE = {
    '협박 대화': '00',
    '갈취 대화': '01',
    '직장 내 괴롭힘 대화': '02',
    '기타 괴롭힘 대화': '03',
    '일반 대화': '04',
}

# -------- Tokenizer & Collate --------
def prepare_tokenizer_and_maxlen():
    cfg = AutoConfig.from_pretrained(CFG.bert_name)
    model_limit = getattr(cfg, "max_position_embeddings", 512)
    CFG.max_len = min(CFG.max_len, model_limit)
    
    tok = AutoTokenizer.from_pretrained(CFG.bert_name, use_fast=True)
    tok.model_max_length = CFG.max_len
    return tok

def make_collate(tokenizer):
    def _fn(batch):
        texts, labels = zip(*batch)
        enc = tokenizer(
            list(texts),
            padding="max_length",
            truncation=True,
            max_length=CFG.max_len,
            return_tensors="pt"
        )
        y = torch.tensor(labels, dtype=torch.long)
        return {"input_ids": enc["input_ids"], "attention_mask": enc["attention_mask"]}, y
    return _fn

class RawDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        y = -1 if self.labels is None else self.labels[i]
        return self.texts[i], y

# -------- Models (수정 버전) --------
class BertCnnHeadV2(nn.Module):
    """BERT + CNN Head (개선 버전)"""
    def __init__(self, bert_name, num_labels, kernels=(2,3,4), channels=128, dropout=0.3):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(bert_name)
        hid = self.backbone.config.hidden_size
        
        # 🔥 Same padding으로 변경
        self.convs = nn.ModuleList([
            nn.Conv1d(hid, channels, k, padding=k//2)
            for k in kernels
        ])
        self.batch_norms = nn.ModuleList([
            nn.BatchNorm1d(channels) for _ in kernels
        ])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(channels * len(kernels), num_labels)
        
    def forward(self, input_ids, attention_mask):
        out = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        H = out.last_hidden_state              # [B, L, H]
        
        # 🔥 개선: Attention mask 직접 곱하기
        H = H * attention_mask.unsqueeze(-1)   # Padding 제거
        
        x = H.transpose(1, 2)                  # [B, H, L]
        
        feats = []
        for conv, bn in zip(self.convs, self.batch_norms):
            h = conv(x)                        # [B, C, L]
            h = bn(h)
            h = F.relu(h)
            # 🔥 간단한 Max Pooling (masking 불필요)
            h = torch.max(h, dim=-1).values    # [B, C]
            feats.append(h)
        
        z = torch.cat(feats, dim=1)
        z = self.dropout(z)
        return self.fc(z)

class TextCNN(nn.Module):
    """TextCNN (원본 유지)"""
    def __init__(self, vocab_size, pad_id, num_labels, emb_dim=300, kernels=(3,4,5), channels=128, dropout=0.2):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        self.convs = nn.ModuleList([nn.Conv1d(emb_dim, channels, k) for k in kernels])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(channels * len(kernels), num_labels)
        
    def forward(self, input_ids, attention_mask=None):
        x = self.emb(input_ids)              # [B, L, E]
        x = x.transpose(1, 2)                # [B, E, L]
        feats = []
        for conv in self.convs:
            h = F.relu(conv(x))              # [B, C, L']
            h = torch.max(h, dim=-1).values  # [B, C]
            feats.append(h)
        z = torch.cat(feats, dim=1)
        z = self.dropout(z)
        return self.fc(z)

# -------- Class Weight --------
def compute_class_weights(y_all, num_classes):
    from sklearn.utils.class_weight import compute_class_weight
    class_weights = compute_class_weight(
        'balanced',
        classes=np.arange(num_classes),
        y=y_all
    )
    # 🔥 기타 괴롭힘(3)과 협박(0) 부스트
    class_weights[3] *= 1.3
    class_weights[0] *= 1.2
    return torch.FloatTensor(class_weights).to(device)

# -------- Train/Eval --------
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    ys, ps = [], []
    for enc, y in loader:
        enc = {k:v.to(device) for k,v in enc.items()}
        y = y.to(device)
        logits = model(**enc)
        ps.append(logits.softmax(1).cpu())
        ys.append(y.cpu())
    P = torch.cat(ps).numpy()
    Y = torch.cat(ys).numpy()
    preds = P.argmax(1)
    f1 = f1_score(Y, preds, average="macro")
    return f1, Y, preds

def train_one_epoch(model, loader, optimizer, scheduler=None, class_weights=None):
    model.train()
    total = 0.0
    pbar = tqdm(loader, ncols=100, leave=False, desc="Training")
    for enc, y in pbar:
        enc = {k:v.to(device) for k,v in enc.items()}
        y = y.to(device)
        
        logits = model(**enc)
        
        # 🔥 Class weight 적용
        if class_weights is not None:
            loss = F.cross_entropy(logits, y, weight=class_weights)
        else:
            loss = F.cross_entropy(logits, y)
        
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        if scheduler: scheduler.step()
        total += loss.item()
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total / max(1, len(loader))

# -------- Main --------
def main():
    print("="*60)
    print(f"🚀 CNN Baseline: {CFG.model_type}")
    print("="*60)
    print(f"Model: {CFG.bert_name}")
    print(f"Max length: {CFG.max_len}")
    print(f"Batch size: {CFG.batch_size}")
    print(f"Epochs: {CFG.epochs}")
    print(f"Class weight: {CFG.use_class_weight}")
    print("="*60 + "\n")
    
    # 1) Data
    train_df = load_train_df(os.path.join(CFG.data_dir, "strage_1.csv"))
    test_df  = load_test_df(os.path.join(CFG.data_dir, "test.csv"))
    
    print(f"Train samples: {len(train_df)}")
    print(f"Test samples: {len(test_df)}\n")

    # labels (고정 순서)
    def order_key(name):
        return list(NAME2CODE.keys()).index(name) if name in NAME2CODE else 999
    labels = sorted(train_df["label"].unique().tolist(), key=order_key)
    lid = {l:i for i,l in enumerate(labels)}
    id2code = {i: NAME2CODE.get(labels[i], f"{i:02d}") for i in range(len(labels))}
    
    print(f"Classes: {len(labels)}")
    for i, label in enumerate(labels):
        count = (train_df["label"] == label).sum()
        print(f"  {i:02d}: {label} ({count}개)")
    print()
    
    y_all = np.array([lid[l] for l in train_df["label"].tolist()])
    texts = train_df["text"].tolist()

    # 2) tokenizer
    tok = prepare_tokenizer_and_maxlen()
    print(f"Tokenizer max_len: {CFG.max_len}\n")

    # 3) stratified split
    sss = StratifiedShuffleSplit(n_splits=1, test_size=CFG.val_ratio, random_state=CFG.seed)
    tr_idx, va_idx = next(sss.split(np.arange(len(texts)), y_all))
    tr_texts, va_texts = [texts[i] for i in tr_idx], [texts[i] for i in va_idx]
    tr_labels, va_labels = y_all[tr_idx].tolist(), y_all[va_idx].tolist()

    # 4) datasets/loaders
    collate = make_collate(tok)
    train_ds = RawDataset(tr_texts, tr_labels)
    val_ds   = RawDataset(va_texts, va_labels)
    test_ds  = RawDataset(test_df["text"].tolist(), None)

    train_loader = torch.utils.data.DataLoader(train_ds, batch_size=CFG.batch_size, shuffle=True,  collate_fn=collate, num_workers=0)
    val_loader   = torch.utils.data.DataLoader(val_ds,   batch_size=CFG.batch_size, shuffle=False, collate_fn=collate, num_workers=0)
    test_loader  = torch.utils.data.DataLoader(test_ds,  batch_size=CFG.batch_size, shuffle=False, collate_fn=collate, num_workers=0)

    # 5) model
    if CFG.model_type == "textcnn":
        model = TextCNN(
            vocab_size=tok.vocab_size,
            pad_id=tok.pad_token_id if tok.pad_token_id is not None else 0,
            num_labels=len(labels),
            emb_dim=300, kernels=(3,4,5), channels=128, dropout=0.2
        )
        lr = CFG.lr_textcnn
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
        scheduler = None
        class_weights = None
    else:
        # 🔥 수정: BertCnnHeadV2 사용
        model = BertCnnHeadV2(
            CFG.bert_name, 
            num_labels=len(labels), 
            kernels=(2,3,4), 
            channels=128, 
            dropout=0.3
        )
        no_decay = ["bias", "LayerNorm.weight"]
        grouped = [
            {"params":[p for n,p in model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay":CFG.weight_decay},
            {"params":[p for n,p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay":0.0},
        ]
        lr = CFG.lr
        optimizer = torch.optim.AdamW(grouped, lr=lr)
        total_steps = math.ceil(len(train_loader) * CFG.epochs)
        warmup = max(1, int(0.1 * total_steps))
        scheduler = torch.optim.lr_scheduler.SequentialLR(
            optimizer,
            schedulers=[
                torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, total_iters=warmup),
                torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=total_steps - warmup),
            ],
            milestones=[warmup]
        )
        
        # 🔥 추가: Class weights
        if CFG.use_class_weight:
            class_weights = compute_class_weights(y_all, len(labels))
            print("Class weights:")
            for i, (label, weight) in enumerate(zip(labels, class_weights)):
                print(f"  {i:02d} {label}: {weight:.4f}")
            print()
        else:
            class_weights = None

    model = model.to(device)

    # 6) train
    print("Training started!\n")
    print("-"*60)
    best_f1, best_state = -1.0, None
    patience_counter = 0
    
    for ep in range(1, CFG.epochs+1):
        tr_loss = train_one_epoch(model, train_loader, optimizer, scheduler, class_weights)
        f1, y_true, y_pred = evaluate(model, val_loader)
        
        print(f"Epoch {ep:2d}/{CFG.epochs}  loss={tr_loss:.4f}  val_f1={f1:.4f}")
        
        if f1 > best_f1:
            best_f1, best_state = f1, {k:v.cpu().clone() for k,v in model.state_dict().items()}
            report = classification_report(y_true, y_pred, target_names=labels, digits=4)
            print(f"  ✅ Best model saved! (F1: {best_f1:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{CFG.early_stopping_patience})")
        
        print("-"*60)
        
        if patience_counter >= CFG.early_stopping_patience:
            print("\n⚠️ Early stopping!")
            break

    # 7) save and predict
    print("\n" + "="*60)
    print("📊 Final Report:")
    print("="*60)
    print(report)
    
    torch.save({"state_dict": best_state, "labels": labels}, os.path.join(CFG.out_dir, f"{CFG.model_type}_best.pt"))
    with open(os.path.join(CFG.out_dir, f"{CFG.model_type}_val_report.txt"), "w", encoding="utf-8") as f:
        f.write(report)

    # load best to predict
    model.load_state_dict(best_state)
    model.eval()
    preds = []
    with torch.no_grad():
        for enc, _ in tqdm(test_loader, ncols=100, desc="Predicting"):
            enc = {k:v.to(device) for k,v in enc.items()}
            logits = model(**enc)
            preds.append(logits.argmax(1).cpu())
    
    pred_ids = torch.cat(preds).numpy()
    
    # 🔥 수정: 예측 ID → 클래스 이름 → 코드
    pred_codes = [NAME2CODE.get(labels[int(i)], f"{int(i):02d}") for i in pred_ids]
    
    # 🔥 수정: conversation 컬럼 추가
    sub = pd.DataFrame({
        "idx": test_df["idx"],
        
        "class": pred_codes
    })
    
    out_path = os.path.join(CFG.out_dir, f"submission_{CFG.model_type}.csv")
    sub.to_csv(out_path, index=False)
    
    print(f"\n✅ Submission saved: {out_path}")
    print(f"Shape: {sub.shape}")
    print(f"\nSample predictions:")
    print(sub.head())

if __name__ == "__main__":
    main()

🚀 CNN Baseline: bertcnn
Model: beomi/KcBERT-base
Max length: 384
Batch size: 16
Epochs: 10
Class weight: True

Train samples: 4750
Test samples: 500

Classes: 5
  00: 협박 대화 (896개)
  01: 갈취 대화 (981개)
  02: 직장 내 괴롭힘 대화 (979개)
  03: 기타 괴롭힘 대화 (1094개)
  04: 일반 (800개)



c:\Users\C\anaconda3\envs\jedi\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenizer max_len: 300

Class weights:
  00 협박 대화: 1.2723
  01 갈취 대화: 0.9684
  02 직장 내 괴롭힘 대화: 0.9704
  03 기타 괴롭힘 대화: 1.1289
  04 일반: 1.1875

Training started!

------------------------------------------------------------


Epoch  1/10  loss=1.2516  val_f1=0.9041
  ✅ Best model saved! (F1: 0.9041)
------------------------------------------------------------


Epoch  2/10  loss=0.3215  val_f1=0.9137
  ✅ Best model saved! (F1: 0.9137)
------------------------------------------------------------


...

Epoch  8/10  loss=0.0009  val_f1=0.9408
  ✅ Best model saved! (F1: 0.9408)
------------------------------------------------------------


Epoch  9/10  loss=0.0006  val_f1=0.9422
  ✅ Best model saved! (F1: 0.9422)
------------------------------------------------------------


Epoch 10/10  loss=0.0001  val_f1=0.9449
  ✅ Best model saved! (F1: 0.9449)
------------------------------------------------------------

📊 Final Report:
              precision    recall  f1-score   support

       협박 대화     0.9265    0.9333    0.9299       135
       갈취 대화     0.9060    0.9184    0.9122       147
 직장 내 괴롭힘 대화     0.9655    0.9524    0.9589       147
   기타 괴롭힘 대화     0.9264    0.9207    0.9235       164
          일반     1.0000    1.0000    1.0000       120

    accuracy                         0.9425       713
   macro avg     0.9449    0.9450    0.9449       713
weighted avg     0.9427    0.9425    0.9426       713



Predicting: 100%|███████████████████████████████████████████████████| 32/32 [00:01<00:00, 18.90it/s]



✅ Submission saved: outputs\submission_bertcnn.csv
Shape: (500, 2)

Sample predictions:
     idx class
0  t_000    01
1  t_001    02
2  t_002    02
3  t_003    04
4  t_004    04
